In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import tensorflow as tf
from helper_functions import *
importTensorflow(memory=4090)
precision()

2.15.0
1 Physical GPUs, 1 Logical GPUs
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3050 Laptop GPU, compute capability 8.6
<Policy "mixed_float16">


In [2]:
train_data, test_data = tf.keras.utils.image_dataset_from_directory('food-101/cuisines/',
                                                        image_size=(512, 512),
                                                        validation_split = 0.3,
                                                        subset = 'both',
                                                        seed = 42,
                                                        batch_size=64,
                                                        shuffle=True)

Found 101000 files belonging to 14 classes.
Using 70700 files for training.
Using 30300 files for validation.


In [3]:
classes = 14
def preprocess_img(image, label):
    return tf.cast(image, tf.float32), label
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.prefetch(tf.data.AUTOTUNE)

In [4]:
from tensorflow.keras import layers
input_shape = (512, 512, 3)
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B3(include_top=False)
base_model.trainable = False
inputs = layers.Input(shape=input_shape, name="input_layer")
# x = data_aug(inputs, training=False)
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Dense(32, activation='relu')(x)
x = layers.Dense(classes)(x)
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
model = tf.keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch:3e-3/10**(epoch/5))

In [5]:
history_101_food_classes_feature_extract = model.fit(train_data,
                                                     epochs=5,
                                                     steps_per_epoch=len(train_data),
                                                     validation_data=test_data,
                                                     validation_steps=int(0.2 * len(test_data)),
                                                     callbacks=[lr_scheduler])

Epoch 1/5


I0000 00:00:1710840064.170330   14197 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1105/1105 [==============================] - 695s 613ms/step - loss: 1.1284 - accuracy: 0.6092 - val_loss: 0.8828 - val_accuracy: 0.7001 - lr: 0.0030
Epoch 2/5
1105/1105 [==============================] - 669s 605ms/step - loss: 0.8498 - accuracy: 0.7061 - val_loss: 0.7918 - val_accuracy: 0.7306 - lr: 0.0019
Epoch 3/5
1105/1105 [==============================] - 669s 605ms/step - loss: 0.7368 - accuracy: 0.7451 - val_loss: 0.7468 - val_accuracy: 0.7488 - lr: 0.0012
Epoch 4/5
1105/1105 [==============================] - 667s 603ms/step - loss: 0.6667 - accuracy: 0.7681 - val_loss: 0.7289 - val_accuracy: 0.7553 - lr: 7.5357e-04
Epoch 5/5
1105/1105 [==============================] - 665s 602ms/step - loss: 0.6190 - accuracy: 0.7851 - val_loss: 0.7116 - val_accuracy: 0.7598 - lr: 4.7547e-04


In [7]:
model.evaluate(test_data)

474/474 [==============================] - 268s 565ms/step - loss: 0.7323 - accuracy: 0.7555


[0.7323012351989746, 0.75547856092453]

In [19]:
model.save('cuisine_feature_extractor')

INFO:tensorflow:Assets written to: cuisine_feature_extractor/assets


INFO:tensorflow:Assets written to: cuisine_feature_extractor/assets


In [20]:
loaded_saved_model = tf.keras.models.load_model('cuisine_feature_extractor')
loaded_saved_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 512, 512, 3)]     0         
                                                                 
 efficientnetv2-b3 (Functio  (None, None, None, 1536   12930622  
 nal)                        )                                   
                                                                 
 pooling_layer (GlobalAvera  (None, 1536)              0         
 gePooling2D)                                                    
                                                                 
 dense (Dense)               (None, 128)               196736    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 14)                1806  

In [21]:
# fine tuning
for i, layer in enumerate(loaded_saved_model.layers):
    print(i, layer.name, layer.trainable)

0 input_layer True
1 efficientnetv2-b3 False
2 pooling_layer True
3 dense True
4 dropout True
5 dense_1 True
6 softmax_float32 True


In [22]:
for i, layer in enumerate(loaded_saved_model.layers[1].layers):
    print(i, layer.name, layer.trainable)

0 input_1 True
1 rescaling False
2 normalization False
3 stem_conv False
4 stem_bn False
5 stem_activation False
6 block1a_project_conv False
7 block1a_project_bn False
8 block1a_project_activation False
9 block1b_project_conv False
10 block1b_project_bn False
11 block1b_project_activation False
12 block1b_drop False
13 block1b_add False
14 block2a_expand_conv False
15 block2a_expand_bn False
16 block2a_expand_activation False
17 block2a_project_conv False
18 block2a_project_bn False
19 block2b_expand_conv False
20 block2b_expand_bn False
21 block2b_expand_activation False
22 block2b_project_conv False
23 block2b_project_bn False
24 block2b_drop False
25 block2b_add False
26 block2c_expand_conv False
27 block2c_expand_bn False
28 block2c_expand_activation False
29 block2c_project_conv False
30 block2c_project_bn False
31 block2c_drop False
32 block2c_add False
33 block3a_expand_conv False
34 block3a_expand_bn False
35 block3a_expand_activation False
36 block3a_project_conv False
37 blo

In [23]:
loaded_saved_model.layers[1].trainable = True
for i, layer in enumerate(loaded_saved_model.layers[1].layers[:-20]):
    layer.trainable = False
for i, layer in enumerate(loaded_saved_model.layers[1].layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 rescaling False
2 normalization False
3 stem_conv False
4 stem_bn False
5 stem_activation False
6 block1a_project_conv False
7 block1a_project_bn False
8 block1a_project_activation False
9 block1b_project_conv False
10 block1b_project_bn False
11 block1b_project_activation False
12 block1b_drop False
13 block1b_add False
14 block2a_expand_conv False
15 block2a_expand_bn False
16 block2a_expand_activation False
17 block2a_project_conv False
18 block2a_project_bn False
19 block2b_expand_conv False
20 block2b_expand_bn False
21 block2b_expand_activation False
22 block2b_project_conv False
23 block2b_project_bn False
24 block2b_drop False
25 block2b_add False
26 block2c_expand_conv False
27 block2c_expand_bn False
28 block2c_expand_activation False
29 block2c_project_conv False
30 block2c_project_bn False
31 block2c_drop False
32 block2c_add False
33 block3a_expand_conv False
34 block3a_expand_bn False
35 block3a_expand_activation False
36 block3a_project_conv False
37 bl

In [24]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                  patience=3)

In [25]:
loaded_saved_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
              metrics=['accuracy'])
history_14_fine_tune = loaded_saved_model.fit(train_data, epochs = 100,
                                initial_epoch = history_101_food_classes_feature_extract.epoch[-1] + 1,
                                validation_data = test_data,
                                steps_per_epoch = len(train_data),
                                validation_steps = int(0.2*len(test_data)),
                                callbacks=[early_stopping])

Epoch 6/100
1105/1105 [==============================] - 722s 642ms/step - loss: 0.5980 - accuracy: 0.7921 - val_loss: 0.6641 - val_accuracy: 0.7769
Epoch 7/100
1105/1105 [==============================] - 710s 642ms/step - loss: 0.5309 - accuracy: 0.8166 - val_loss: 0.6522 - val_accuracy: 0.7846
Epoch 8/100
1105/1105 [==============================] - 709s 642ms/step - loss: 0.4644 - accuracy: 0.8398 - val_loss: 0.6457 - val_accuracy: 0.7859
Epoch 9/100
1105/1105 [==============================] - 710s 642ms/step - loss: 0.4030 - accuracy: 0.8611 - val_loss: 0.6580 - val_accuracy: 0.7871
Epoch 10/100
1105/1105 [==============================] - 709s 641ms/step - loss: 0.3377 - accuracy: 0.8829 - val_loss: 0.6725 - val_accuracy: 0.7911
Epoch 11/100
1105/1105 [==============================] - 713s 645ms/step - loss: 0.2824 - accuracy: 0.9022 - val_loss: 0.6688 - val_accuracy: 0.7984


In [26]:
loaded_saved_model.evaluate(test_data)

474/474 [==============================] - 270s 569ms/step - loss: 0.6986 - accuracy: 0.7963


[0.6985512971878052, 0.7963036298751831]

In [27]:
loaded_saved_model.save("cuisine_79_6.h5")

/home/ujjwal/miniconda3/envs/tf2.15/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
